<a href="https://colab.research.google.com/github/XiaoshanZhou624/velocity-replay/blob/main/Contrastive_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch.optim as optim
import torch.nn as nn
from scipy.ndimage import gaussian_filter1d

# Gaussian smoothing function
def gaussian_smooth(data, sigma=1):
    return gaussian_filter1d(data, sigma=sigma)

# Autonomous system dynamics simulation
def generate_system_speeds(time, dt, a=0.5, initial_speed=0):
    system_speeds = np.zeros_like(time)
    system_speeds[0] = initial_speed
    for i in range(1, len(time)):
        system_speeds[i] = system_speeds[i-1] + a * dt
    return system_speeds

# Generate human input based on Bayesian updating
def generate_human_preferred_speeds(time, system_speeds, initial_speed=0):
    human_changes = np.random.normal(0, 0.1, size=time.shape)
    human_input = np.cumsum(human_changes)
    human_input = 2 * (human_input - np.min(human_input)) / (np.max(human_input) - np.min(human_input)) - 1
    human_input = np.abs(human_input)  # Make all values positive

    human_preferred_speeds = np.zeros_like(time)
    human_preferred_speeds[0] = initial_speed + human_input[0]
    for i in range(1, len(time)):
        prior_mean = 0.5 * system_speeds[i] + 0.5 * human_preferred_speeds[i-1]
        human_preferred_speeds[i] = (2 * prior_mean + human_input[i]) / 2

    return human_preferred_speeds

# Time setup
def generate_time(T=10, dt=0.1):
    return np.arange(0, T + dt, dt)


# Modified Dataset Class with Sliding Window
class SimulatedSpeedDataset(Dataset):
    def __init__(self, window_size=8):
        self.time = generate_time()
        self.system_speeds = generate_system_speeds(self.time, dt=0.1)
        self.human_preferred_speeds = generate_human_preferred_speeds(self.time, self.system_speeds)
        self.window_size = window_size

    def __len__(self):
        return len(self.time) - self.window_size

    def __getitem__(self, idx):
        # Get window of speeds
        system_speed_window = self.system_speeds[idx:idx + self.window_size]
        human_speed_window = self.human_preferred_speeds[idx:idx + self.window_size]

        # Apply Gaussian smoothing to the windows
        system_speed_window = gaussian_smooth(system_speed_window, sigma=1)
        human_speed_window = gaussian_smooth(human_speed_window, sigma=1)

        # Generate positive pair (next window in sequence)
        positive_idx = (idx + 1) % (len(self.time) - self.window_size)
        positive_window = self.human_preferred_speeds[positive_idx:positive_idx + self.window_size]
        positive_window = gaussian_smooth(positive_window, sigma=1)

        # Generate negative pair (random window)
        negative_idx = np.random.randint(0, len(self.time) - self.window_size)
        negative_window = self.human_preferred_speeds[negative_idx:negative_idx + self.window_size]
        negative_window = gaussian_smooth(negative_window, sigma=1)

        # Convert to torch tensors
        return (torch.tensor(system_speed_window, dtype=torch.float32),
                torch.tensor(human_speed_window, dtype=torch.float32),
                torch.tensor(positive_window, dtype=torch.float32),
                torch.tensor(negative_window, dtype=torch.float32))


class SiameseNetworkGRU(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=1, window_size=3):
        super(SiameseNetworkGRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)  # GRU Layer
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 16),  # Reduce the hidden size output
            nn.ReLU(),
            nn.Linear(16, 8)
        )

    def forward(self, input1, input2):
        # GRU expects input of shape (batch_size, seq_len, input_size)
        output1, _ = self.gru(input1)
        output2, _ = self.gru(input2)

        # Use the last output of the GRU (output1[:,-1,:])
        output1 = self.fc(output1[:, -1, :])
        output2 = self.fc(output2[:, -1, :])

        return output1, output2

    def embedding(self, x):
        output, _ = self.gru(x)
        return self.fc(output[:, -1, :])


def contrastive_loss(output1, output2, label, margin=1.0, eps=1e-6):
    # Compute the Euclidean distance between two windows of embeddings
    euclidean_distance = ((output1 - output2).pow(2).sum(1) + eps).sqrt()

    # Calculate the loss for positive and negative pairs
    loss_positive = (1 - label) * euclidean_distance.pow(2)
    loss_negative = label * torch.clamp(margin - euclidean_distance, min=0.0).pow(2)
    loss_contrastive = torch.mean(loss_positive + loss_negative)

    return loss_contrastive

def train(model, dataloader, epochs=10, lr=0.01, window_size=4):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) # weight_decay=1e-6
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.99, patience=5)
    model.train()
    epoch_losses = []

    for epoch in range(epochs):
        total_loss = 0
        num_batches = 0

        for system_speed_window, human_speed_window, positive_window, negative_window in dataloader:
            optimizer.zero_grad()

            # Reshape to (batch_size, window_size, input_size=1)
            human_speed_window = human_speed_window.unsqueeze(-1)  # (batch_size, window_size) -> (batch_size, window_size, 1)
            system_speed_window = system_speed_window.unsqueeze(-1)
            positive_window = positive_window.unsqueeze(-1)
            negative_window = negative_window.unsqueeze(-1)

            # Compute embeddings for windows
            human_emb, system_emb = model(human_speed_window, system_speed_window)
            positive_emb, _ = model(positive_window, system_speed_window)
            negative_emb, _ = model(negative_window, system_speed_window)

            # Calculate loss for positive and negative pairs
            positive_loss = contrastive_loss(human_emb, positive_emb, torch.zeros(human_emb.size(0), dtype=torch.float32))
            negative_loss = contrastive_loss(human_emb, negative_emb, torch.ones(human_emb.size(0), dtype=torch.float32))

            loss = positive_loss + negative_loss
            loss.backward()
            # Apply gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()
            num_batches += 1

        average_loss = total_loss / num_batches
        epoch_losses.append(average_loss)

        print(f"Epoch {epoch+1}, Average Loss: {average_loss}")

        scheduler.step(average_loss)

    return epoch_losses

# Initialize dataset and model
dataset = SimulatedSpeedDataset()
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
model = SiameseNetworkGRU()
# train(model, dataloader)

# Train the model
epoch_losses = train(model, dataloader, epochs=100)

# Plotting the training loss
plt.figure(figsize=(10, 5))
plt.plot(epoch_losses, label='Training Loss')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.legend()
plt.show()

def visualize_embeddings(model, dataloader):
    model.eval()
    embeddings, labels = [], []

    with torch.no_grad():
        for system_speed, human_speed, _, _ in dataloader:
            # Reshape to 3D tensor: (batch_size, sequence_length=1, input_size=1)
            system_speed = system_speed.unsqueeze(-1)  # Adding the third dimension
            human_speed = human_speed.unsqueeze(-1)

            # Get embeddings for system speed
            emb = model.embedding(system_speed).squeeze(0).numpy()  # Remove batch dimension
            embeddings.append(emb)
            labels.append('System Speed')

            # Get embeddings for human speed
            emb = model.embedding(human_speed).squeeze(0).numpy()  # Remove batch dimension
            embeddings.append(emb)
            labels.append('Human Speed')

    # Stack all embeddings and reduce dimensions using t-SNE
    embeddings = np.vstack(embeddings)
    embeddings_reduced = TSNE(n_components=2).fit_transform(embeddings)  # Using t-SNE for dimensionality reduction

    # Plot the reduced embeddings
    plt.figure(figsize=(10, 5))
    for i, label in enumerate(np.unique(labels)):
        indices = [j for j, x in enumerate(labels) if x == label]
        plt.scatter(embeddings_reduced[indices, 0], embeddings_reduced[indices, 1], label=label, alpha=0.5)
    plt.legend()
    plt.title('t-SNE of System and Human Speed Embeddings')
    plt.xlabel('t-SNE Feature 1')
    plt.ylabel('t-SNE Feature 2')
    plt.show()

# Assuming `dataset` is your SimulatedSpeedDataset instance
visualize_embeddings(model, DataLoader(dataset, batch_size=1, shuffle=False))


class VelocityRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(VelocityRegressionModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 16),  # Make sure input_dim matches the embedding size
            nn.ReLU(),
            nn.Linear(16, 1)  # Output is the predicted velocity (1 value)
        )

    def forward(self, x):
        return self.fc(x)

def train_regression_model(model, dataloader, regression_model, num_epochs=100, lr=0.001):
    optimizer = torch.optim.Adam(regression_model.parameters(), lr=lr)
    criterion = nn.MSELoss()  # Mean Squared Error for regression
    epoch_losses = []  # List to store loss for each epoch

    for epoch in range(num_epochs):
        total_loss = 0
        num_batches = 0

        for system_speed, human_speed, _, _ in dataloader:
            optimizer.zero_grad()

            # Prepare inputs
            system_speed = system_speed.unsqueeze(-1)
            human_speed = human_speed.unsqueeze(-1)

            # Get embeddings from the model
            system_emb = model.embedding(system_speed).squeeze(0)
            human_emb = model.embedding(human_speed).squeeze(0)

            # Predict velocities from embeddings using the regression model
            predicted_system_speed = regression_model(system_emb)
            predicted_human_speed = regression_model(human_emb)

            # Loss between predicted and actual velocities
            loss_system = criterion(predicted_system_speed, system_speed.squeeze(-1))
            loss_human = criterion(predicted_human_speed, human_speed.squeeze(-1))
            loss = loss_system + loss_human

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            num_batches += 1

        # Calculate average loss for the epoch
        avg_loss = total_loss / num_batches
        epoch_losses.append(avg_loss)  # Store the average loss for this epoch

        print(f'Epoch {epoch + 1}, Loss: {avg_loss}')

    # Plot the loss over epochs
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), epoch_losses, label='Training Loss', color='blue')
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


def visualize_velocity_comparison(model, regression_model, dataloader):
    model.eval()
    regression_model.eval()
    actual_system_speeds, predicted_system_speeds = [], []
    actual_human_speeds, predicted_human_speeds = [], []

    with torch.no_grad():
        for system_speed, human_speed, _, _ in dataloader:
            # Prepare inputs
            system_speed = system_speed.unsqueeze(-1)
            human_speed = human_speed.unsqueeze(-1)

            # Get embeddings from the model
            system_emb = model.embedding(system_speed).squeeze(0)
            human_emb = model.embedding(human_speed).squeeze(0)

            # Predict velocities from embeddings using the regression model
            predicted_system_speed = regression_model(system_emb)
            predicted_human_speed = regression_model(human_emb)

            # Store actual and predicted values for visualization
            actual_system_speeds.append(system_speed.squeeze(-1).numpy())
            predicted_system_speeds.append(predicted_system_speed.numpy())
            actual_human_speeds.append(human_speed.squeeze(-1).numpy())
            predicted_human_speeds.append(predicted_human_speed.numpy())

    # Convert to numpy arrays
    actual_system_speeds = np.concatenate(actual_system_speeds)
    predicted_system_speeds = np.concatenate(predicted_system_speeds)
    actual_human_speeds = np.concatenate(actual_human_speeds)
    predicted_human_speeds = np.concatenate(predicted_human_speeds)

    # Plot System Velocity
    plt.figure(figsize=(10, 5))
    plt.plot(actual_system_speeds, label='Actual System Speed', color='blue')
    plt.plot(predicted_system_speeds, label='Predicted System Speed', color='red', linestyle='--')
    plt.title('System Speed: Actual vs Predicted')
    plt.xlabel('Time')
    plt.ylabel('Speed')
    plt.legend()
    plt.show()

    # Plot Human-Preferred Velocity
    plt.figure(figsize=(10, 5))
    plt.plot(actual_human_speeds, label='Actual Human Speed', color='blue')
    plt.plot(predicted_human_speeds, label='Predicted Human Speed', color='red', linestyle='--')
    plt.title('Human-Preferred Speed: Actual vs Predicted')
    plt.xlabel('Time')
    plt.ylabel('Speed')
    plt.legend()

    # Save the figure if a path is provided
    save_path="t-SNE.png"
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"Visualization saved as {save_path}")

    plt.show()

def smooth_connect(predicted_human_speeds, smooth_length=20):
    # Create a linear interpolation from 0 to the value at the 20th point
    transition_values = np.linspace(0, predicted_human_speeds[smooth_length], smooth_length)

    # Replace the first smooth_length points with the transition values
    predicted_human_speeds[:smooth_length] = transition_values

    # Return the modified array
    return predicted_human_speeds


def visualize_velocity_comparison_replay(model, regression_model, dataloader):
    model.eval()
    regression_model.eval()
    actual_human_speeds, predicted_human_speeds = [], []

    with torch.no_grad():
        for system_speed, human_speed, _, _ in dataloader:
            # Prepare inputs
            human_speed = human_speed.unsqueeze(-1)

            # Get embeddings from the model
            human_emb = model.embedding(human_speed).squeeze(0)

            # Predict velocities from embeddings using the regression model
            predicted_human_speed = regression_model(human_emb)

            # Store actual and predicted values for visualization
            actual_human_speeds.append(human_speed.squeeze(-1).numpy())
            predicted_human_speeds.append(predicted_human_speed.numpy())

    # Convert to numpy arrays
    actual_human_speeds = np.concatenate(actual_human_speeds)
    predicted_human_speeds = np.concatenate(predicted_human_speeds)

    # Apply the smooth transition to predicted human speeds
    predicted_human_speeds = smooth_connect(predicted_human_speeds, smooth_length=20)

    # Plot Human-Preferred Velocity
    plt.figure(figsize=(10, 5))
    plt.plot(actual_human_speeds, label='Actual Human Speed', color='blue')
    plt.plot(predicted_human_speeds, label='Human Predicted Speed', color='red', linestyle='--')
    plt.title('Human-Preferred Speed: Actual vs Predicted')
    plt.xlabel('Time')
    plt.ylabel('Speed')
    plt.legend(loc='upper left')  # Simplified legend with only two items

    # Save the figure if a path is provided
    save_path="velocity_reproduce.png"
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"Visualization saved as {save_path}")

    plt.show()


# Initialize the regression model with the correct embedding size
embedding_size = 8  # Set this to the size of the embeddings produced by the model
regression_model = VelocityRegressionModel(input_dim=embedding_size)

# Train the regression model
train_regression_model(model, DataLoader(dataset, batch_size=1, shuffle=False), regression_model)

# visualize_velocity_comparison(model, regression_model, DataLoader(dataset, batch_size=1, shuffle=False))
# visualize_velocity_comparison_replay(model, regression_model, DataLoader(dataset, batch_size=1, shuffle=False))
visualize_velocity_comparison(model, regression_model, DataLoader(dataset, batch_size=1, shuffle=False))
